# VAns—Variable Ansatz

Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.

## Overview

Variational Quantum Algorithms (VQA) are about to tune parameters of quantum circuits to minimize an objective function of interest. The commonly used VQAs, like Variational Quantum Eigensolvers (VQE) and Quantum Approximate Optimization Algorithm (QAOA), perform optimization using a user-defined ansatz with fixed structure. However, if the ansatz is too simple or shallow, the expressivity of the circuit will not be insufficient to get the optimal value for the objective function. On the other side, if the ansatz is overly complicated or long, we may encounter barren plateau effect, which impedes us from obtaining the global minimum value. Thus, a circuit structure design search algorithm will be helpful to find the appropriate circuit for a specific task.

In this tutorial, we will discuss a circuit architecture design search algorithm called Variable ansatz (VAns) [1]. Starting with an initial circuit, VAns keeps inserting and deleting gates during the optimization process in order to minimize the loss function as well as keep the circuit shallow. We will use this method to accomplish an adjusted version of VQE task and compare the resulting circuit with the original VQE circuit.

## Pipeline

VAns consists of the following steps:  
1. Start with an initial circuit. Run the inner optimization, which is just the original VQE process, and get the optimal value. 
2. Randomly choose a block from the 'pool' (As the following figure shows, each block only consists of $R_y$, $R_z$, and $CNOT$ gates), and insert the block at the end of circuit. Qubits that the block is applied on are also uniformly sampled. The parameters of the inserted gates are initialized to $0$ so that the inserted circuit is equivalent to identity. 

![Inserting Blocks](./figures/vans-fig-blocks.png)

3. Simplify the circuit according to the following rules. Run the optimization process, and get the optimal value of loss function. Compare the loss value to the previously stored one, decide whether to accept the new circuit or not according to a pre-defined threshold. If the circuit is accepted, remove gates that do not lower the loss, set the circuit as the current circuit and store the corresponding loss value.

![Simplification rules](./figures/vans-fig-rules.png)

4. Repeat steps 2-3 for chosen number of iterations, output the resulting circuit and the optimal loss value.


## Paddle Quantum Implementation

We consider to get the ground state energy of Hydrogen molecule using [Variation Quantum Eigensolver (VQE)](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html) together with VAns to optimize the circuit structure as well. First, we import the required packages.

In [1]:
import paddle
import paddle_quantum
import paddle_quantum.qchem as qchem
import numpy as np
from paddle_quantum.ansatz import Circuit
from paddle_quantum.ansatz.vans import Inserter, Simplifier, VAns, cir_decompose
from paddle_quantum.hamiltonian import Hamiltonian
import warnings

warnings.filterwarnings("ignore")
np.random.seed(11)
paddle.seed(11)

Before using the VAns algorithm, we need to get the Hamiltonian for Hydrogen. Details can be found in the VQE tutorial.

In [ ]:
geo = qchem.geometry(structure=[["H", [-0.0, 0.0, 0.0]], ["H", [-0.0, 0.0, 0.74]]])
# Save molecule information in to variable molecule, including one-body integrations, molecular and Hamiltonian
molecule = qchem.get_molecular_data(
    geometry=geo,
    basis="sto-3g",
    charge=0,
    multiplicity=1,
    method="fci",
    if_save=True,
    if_print=True,
)
# Recall Hamiltonian
molecular_hamiltonian = qchem.spin_hamiltonian(
    molecule=molecule,
    filename=None,
    multiplicity=1,
    mapping_method="jordan_wigner",
)
# n is the number of qubits
n = molecular_hamiltonian.n_qubits

Then we define the loss function, which is simply the expectation value of the Hamiltonian.

In [3]:
# Define the loss function
def loss_func(cir: Circuit, H: Hamiltonian) -> paddle.Tensor:
    return cir().expec_val(H)

Next, we also need to set some training hyper-parameters before training.

In [4]:
EPSI = 0.001 # set the epsilon
IR = 1 # set the insertion rate
ITERI = 120 # set the number of iterations used for VQE
ITERO = 5 # set the number of iterations used for structure optimization
LR = 0.1 # set the learning rate
T = 0.01 # set the threshold
A = 100 # set the accept wall
IS0 = True # if the initial state is |0>, set to true

paddle_quantum.set_backend('state_vector') # set the backend to state vector

In [5]:
# initialize the framework
vans = VAns(n, loss_func, molecular_hamiltonian,
           epsilon=EPSI,
           insert_rate=IR,
           iter=ITERI,
           iter_out=ITERO,
           LR=LR,
           threshold=T,
           accept_wall=A,
           zero_init_state=IS0)

### Initial Circuit

We give a default initial circuit with uniformly sampled parameters. Then run the optimization process to get the optimal parameters in this initial circuit. The optimization process is the same as in the original VQE. Hyperparameters **iter** and **LR** are used in this process.The obtained loss function and the circuit with optimized parameters will be the initial point for the architecture optimization process. Note that the circuit is simplified when we decided to use this optimized circuit as the current starting point. Details of simplification can be found later.

In [6]:
# Optimize the initial circuit
itr_loss = vans.optimization(vans.cir) 

# Update the loss
vans.loss = itr_loss

# Print out the current circuit
print("Current circuit is:\n" + str(vans.cir))

iter: 20 loss: [-0.99608546]
iter: 40 loss: [-1.0926807]
iter: 60 loss: [-1.1136395]
iter: 80 loss: [-1.1163319]
iter: 100 loss: [-1.1167175]
iter: 120 loss: [-1.1167547]
Current circuit is:
--Rx(3.144)----Rz(3.512)----*----Rx(6.286)----Rz(2.441)-------------------------------------------------------------x--
                            |                                                                                       |  
----------------------------x----Rx(4.249)----Rz(3.141)----Rx(4.246)----Rz(5.477)----*------------------------------|--
                                                                                     |                              |  
--Rx(0.001)----Rz(5.499)----*--------------------------------------------------------x----Rx(3.141)----Rz(2.688)----|--
                            |                                                                                       |  
----------------------------x----Rx(0.003)----Rz(2.362)----Rx(0.003)----Rz(1.500)--------

### Insertion

Now, we randomly choose a block from the 'pool', and insert the block at the end of circuit. Qubits that the block is applied on are also uniformly sampled. The parameters of the inserted set of gates are initialized to $0$ so that the new circuit can take the advantage of the previously optimized circuit since they act as identity. The code below inserts sets of gates into the circuit. The number of sets of gates depends on the hyperparameter **insert_rate**. Another hyperparameter **epsilon** is used to determine the variance from $0$ of initial parameters.

In [7]:
# Insert indentity blocks into the current circuit, before doing so, need to
# decompose the circuit from layers to gates
new_cir = cir_decompose(vans.cir)
new_cir = Inserter.insert_identities(new_cir, vans.insert_rate, vans.epsilon)

# Print the updated circuit
print("Circuit after insertion:\n" + str(new_cir))

Circuit after insertion:
--Rx(3.144)----Rz(3.512)----*----Rx(6.286)----Rz(2.441)----------------------------------------------------------------------------------------------------x--
                            |                                                                                                                              |  
----------------------------x----Rz(0.000)----Rx(0.000)----Rx(-0.00)----Rx(4.249)----Rz(3.141)----Rx(4.246)----Rz(5.477)----*------------------------------|--
                                                                                                                            |                              |  
--Rx(0.001)----Rz(5.499)----*-----------------------------------------------------------------------------------------------x----Rx(3.141)----Rz(2.688)----|--
                            |                                                                                                                              |  
---------------------

### Simplification

Then we will simplify the new circuit. Rules are pretty simple:
1. Combine consecutive $CNOT$ gates.
2. Combine rotations gates.
3. Remove $CNOT$ gates and $Rz$ gates in the front of the circuit if the initial state is $|0\rangle$.
4. Commute rotation gates and $CNOT$ gates if the circuit can be further simplified in doing so.

Then we run the parameter optimization process again and get the new minimized value for the loss function. If the new value is smaller than the previous value or the difference between them is smaller than a value that depends on the hyperparameter **accept_wall**, we accpet the new circuit as the current circuit. After accepting the new circuit, we will remove gates that do not lower the loss or lower the loss within a **threshold**.

In [8]:
# Simplify the updated circuit according to the simplification rules
new_cir = Simplifier.simplify_circuit(new_cir, vans.zero_init_state)

# Print the simplied circuits
print(new_cir)

# Then optimize the simplified circuits
itr_loss = vans.optimization(new_cir)

# Calculate the change of loss
relative_diff = (itr_loss - vans.loss) / np.abs(itr_loss)

# If the loss is decreased or increased within a threshold, accept the new circuit
if relative_diff <= 0 or np.random.random() <= np.exp(
    -relative_diff * vans.accept_wall
):
    print("Accpet the new circuit!")

    # Remove gates that do not lower the loss
    new_cir = vans.delete_gates(new_cir, itr_loss)
    new_cir = Simplifier.simplify_circuit(new_cir, vans.zero_init_state)
    itr_loss = loss_func(new_cir, *vans.loss_args)
    vans.loss = itr_loss
else:
    print("Decline the new circuit!")

--Rx(3.144)----Rz(3.512)----*----Rx(6.286)----Rz(2.441)------------------------------------------------x--
                            |                                                                          |  
----------------------------x----Rz(4.491)----Rx(4.470)----Rz(8.712)----*------------------------------|--
                                                                        |                              |  
--Rx(0.001)----Rz(5.499)----*-------------------------------------------x----Rx(3.141)----Rz(2.688)----|--
                            |                                                                          |  
----------------------------x----Rz(3.604)----Rx(5.126)----Rz(4.462)-----------------------------------*--
                                                                                                          
iter: 20 loss: [-1.1121466]
iter: 40 loss: [-1.1123195]
iter: 60 loss: [-1.1158162]
iter: 80 loss: [-1.1166465]
iter: 100 loss: [-1.1167465]
ite

Then we update the current circuit to the new circuit.

In [9]:
# Update the current circuit
vans.cir = new_cir

print("The current circuit:\n" + str(vans.cir))

The current circuit:
--Rx(3.142)----*----------------------x--
               |                      |  
---------------x----*-----------------|--
                    |                 |  
--------------------x----Rx(3.142)----|--
                                      |  
--------------------------------------*--
                                         


The insertion and simplification together forms one iteration for the architecture optimization process. The number of iterations is determined by the hyperparameter **iter_out**. 

### Simple version

While you can customize your own optimization process by adjusting insertion and simplification processes and implementing the training process, Paddle Quantum provides an elegant version of VAns. You can complete the architecture optimization all at once.

In [54]:
# optimization process all at once
vans.train()

Out iteration 1 for structure optimization:
iter: 20 loss: [-1.1167217]
iter: 40 loss: [-1.1166946]
iter: 60 loss: [-1.1167494]
iter: 80 loss: [-1.116759]
iter: 100 loss: [-1.1167591]
iter: 120 loss: [-1.1167595]
 Current loss: [-1.1167595]
 Current cir:
--Rx(3.142)----*----------------------x--
               |                      |  
---------------x----*-----------------|--
                    |                 |  
--------------------x----Rx(3.142)----|--
                                      |  
--------------------------------------*--
                                          

Out iteration 2 for structure optimization:
iter: 20 loss: [-1.099879]
iter: 40 loss: [-1.1144477]
iter: 60 loss: [-1.1166729]
iter: 80 loss: [-1.1167666]
iter: 100 loss: [-1.1207733]
iter: 120 loss: [-1.1371529]
     accpet the new circuit!
     start deleting gates
         Deletion: reject deletion
         Deletion: accept deletion with acceptable loss
         Deletion: reject deletion
         Dele

The final circuit we got from VAns is

In [55]:
print("Final circuit:\n" + str(vans.cir))
print("Final loss:\n" + str(vans.loss))

Final circuit:
--Rx(3.141)----*------------------------------------------------------------------x--
               |                                                                  |  
---------------x----*----Rx(-0.22)----Rz(1.497)----*--------*---------------------|--
                    |                              |        |                     |  
--------------------|------------------------------|--------x--------Rx(3.141)----|--
                    |                              |                              |  
--------------------x------------------------------x----Rz(3.068)-----------------*--
                                                                                     
Final loss:
-1.1372839212417603


## Comparison with original VQE

The circuit we got using VAns consists of only 5 parameters and the depth of the circuit is 9. The minimized loss value is $-1.13728392$ Ha. Comparing to the fixed ansatz used in the original VQE tutorial, where the circuit consists of 12 parameters and with depth 11, VAns reduces the number of parameters needed while keeps the circuit shallower. 

_______

## References

[1] Bilkis, M., et al. "A semi-agnostic ansatz with variable structure for quantum machine learning." [arXiv preprint arXiv:2103.06712 (2021).](https://arxiv.org/abs/2103.06712)